<a href="https://colab.research.google.com/github/jburyk/Swiss_history_semantic_search/blob/main/Swiss_history_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required packages


In [1]:
!pip install -qU langchain openai tiktoken pinecone-client[grpc] apache_beam mwparserfromhell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.2/516.2 kB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [8]:
from getpass import getpass
OPENAI_API_KEY = getpass('Enter OpenAI key: ')
PINECONE_API_KEY = getpass('Enter Pinecone key: ')

Enter OpenAI key: ··········
Enter Pinecone key: ··········


In [2]:
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

inputText = open_file('Switzerland.txt')
inputText = inputText.replace('\n', ' ').replace('\r', '')


In [3]:
import tiktoken

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tokenizer = tiktoken.get_encoding('cl100k_base')

In [4]:
tiktoken_len(inputText)

16674

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=150,
        chunk_overlap=20,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )

chunks = text_splitter.split_text(inputText)

In [6]:
chunks[1]

'franchises. Unlike all similar confederations (e.g., the Hanseatic and Swabian leagues) and despite endemic internal strife, especially after the Reformation in the 16th century, the Swiss Confederation survived the formation of (princely) modern states without adapting to it. With Venice, Genoa, and the Netherlands, the confederation formed the republican exception in Europe, and it'

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(   
    openai_api_key=OPENAI_API_KEY
)

In [11]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed.embed_documents(texts)
len(res), len(res[0])


(2, 1536)

In [12]:
import pinecone

# find API key in console at app.pinecone.io

# find ENV (cloud region) next to API key in console
YOUR_ENV = input("Pinecone environment: ")

index_name = 'langchain-retrieval-augmentation'
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='asia-southeast1-gcp'
)

# we create a new index
pinecone.create_index(
    name=index_name,
    metric='dotproduct',
    dimension=len(res[0])  # 1536 dim of text-embedding-ada-002
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Pinecone environment: asia-southeast1-gcp


In [28]:
index = pinecone.GRPCIndex(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 200}},
 'total_vector_count': 200}

In [27]:
from tqdm.auto import tqdm
from uuid import uuid4
from pprint import pprint

batch_limit = 100

texts = []
metadatas = []

for i, text in enumerate(chunks):
    # first get metadata fields for this record
    
    # now we create chunks from the record text
    
    # create individual metadata dicts for each chunk
    record_metadata = {"chunk": i, "text": text }   
#    pprint(text)
#    pprint(record_metadata)
    # append these to current batches
    texts.append(text)
    metadatas.append(record_metadata)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        pprint(texts[0])        
        pprint(metadatas[0])        
        ids = [str(uuid4()) for _ in range(len(texts))]        
        #pprint(metadatas[0])
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

('Switzerland’s history is one of a medieval defensive league formed during a '
 'time and in an area lacking imperial authority. The different cantons '
 '(traditionally called Orte in German) were to a large extent independent '
 'states that remained united through the shared defense of liberty, which was '
 'understood as the protection of imperial privileges and franchises. Unlike '
 'all similar confederations (e.g., the')
{'chunk': 0,
 'text': 'Switzerland’s history is one of a medieval defensive league formed '
         'during a time and in an area lacking imperial authority. The '
         'different cantons (traditionally called Orte in German) were to a '
         'large extent independent states that remained united through the '
         'shared defense of liberty, which was understood as the protection of '
         'imperial privileges and franchises. Unlike all similar '
         'confederations (e.g., the'}
('which had joined to form theChristian Union, but little fig

In [29]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [38]:
query = "food"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='political power tinged with nepotism was characteristic of the bourgeois capitalism of Escher’s time.  Another effect of the quick development of rail (and maritime) transportation was the growing importation of cheap cereals, which plunged Swiss agriculture into crisis. In 1850 more than half the grain consumed was produced domestically, but by 1914 that measure stood at about', metadata={'chunk': 168.0}),
 Document(page_content='vegetables were brought from the south. The grapevine was introduced despite attempts by Roman legislators to prevent wine from being produced north of the Alps. To facilitate increasing exports of wheat, cattle, and cheese, as well as to provide better lines of communication for military purposes, roads connecting Rome and the northern outposts of the empire were extended and improved', metadata={'chunk': 19.0}),
 Document(page_content='prey, mainly reindeer and bear, into the high mountain valleys. Toward the end of the Würm, about 1